In [3]:
from pprint import pprint as pp
pp("done")

'done'


In [3]:
# global
from typing import List, Optional, Type, Union
import builtins

import ivy
import ivy_models
from ivy_models.inceptionnetv3.layers import Inception_BasicConv2d, Inception_InceptionAux, Inception_InceptionE, Inception_InceptionD, Inception_InceptionC, Inception_InceptionD, Inception_InceptionB, Inception_InceptionA
from ivy_models.base import BaseSpec, BaseModel

ModuleNotFoundError: No module named 'ivy_models.inceptionnetv3'

# test_inceptionv3.py

In [4]:
import os
import random
import ivy
import pytest
import numpy as np
from ivy_models_tests import helpers
# from ivy_models.inceptionv3 import inceptionNet_v3
import jax
jax.config.update("jax_enable_x64", False)


load_weights = random.choice([False, True])
model = inceptionNet_v3(pretrained=load_weights)
v = ivy.to_numpy(model.v)

In [11]:
@pytest.mark.parametrize("data_format", ["NHWC", "NCHW"])
def test_inceptionV3_tiny_img_classification(device, fw, data_format):
    """Test AlexNet image classification."""
    num_classes = 1000
    batch_shape = [1]
    this_dir = os.path.dirname(os.path.realpath(__file__))
    pp(f"done 1/5 ------------------------------------------------")
    
    # Load image
    img = helpers.load_and_preprocess_img(
        os.path.join(this_dir, "..", "..", "images", "dog.jpg"),
        256,
        224,
        data_format=data_format,
        to_ivy=True,
    )
    pp(f"done 2/5 ------------------------------------------------")
    
    model.v = ivy.asarray(v)
    logits, _ = model(img, data_format=data_format)
    pp(f"done 3/4 ------------------------------------------------")
    
    # Cardinality test
    assert logits.shape == tuple([ivy.to_scalar(batch_shape), num_classes])
    pp(f"done 4/5 ------------------------------------------------")
    
    
    # Value test
    if load_weights:
        np_out = ivy.to_numpy(logits[0])
        true_indices = np.array([258, 270, 279])
        calc_indices = np.argsort(np_out)[-5:][::-1]
        assert np.array_equal(true_indices, calc_indices)

        true_logits = np.array([9.9990e-01, 8.3909e-05, 1.1693e-05])
        calc_logits = np.take(np_out, calc_indices)
        assert np.allclose(true_logits, calc_logits, rtol=1)
    pp(f"done 5/5 ------------------------------------------------")

In [14]:
! pytest test_inceptionV3_tiny_img_classification --backend torch

============================= test session starts ==============================
platform linux -- Python 3.10.0, pytest-7.4.0, pluggy-1.2.0
rootdir: /ivy_models
plugins: anyio-3.7.1, metadata-3.0.0, json-report-1.5.0, hypothesis-6.82.3
collected 0 items                                                              

============================ no tests ran in 0.01s =============================
ERROR: file or directory not found: test_inceptionV3_tiny_img_classification

